In [ ]:

import numpy as np
import pandas as pd
import sklearn
import nltk
from nltk import WhitespaceTokenizer


Load preprocessed data


In [ ]:
def load_tweets(file_path):
    tweets = list()
    with open(file_path, 'r', encoding='utf-8') as preprocessed_tweets:
        for tweet in preprocessed_tweets :
            tweets.append(tweet.rstrip('\n'))
    return tweets

In [ ]:
    train_pos_tweets = load_tweets('/content/processed_pos_tweets_non_transformer.txt')
    train_neg_tweets = load_tweets('/content/processed_neg_tweets_non_transformer.txt')
    test_tweets = load_tweets('/content/processed_test_tweets_non_transformer.txt')
    print("Tweets loaded")

Tweets loaded


Convert to Dataframe

In [ ]:
    from sklearn.model_selection import train_test_split
    from sklearn.utils import shuffle
    seed =12222
    train_neg_labels = [0] * len(train_neg_tweets)
    train_pos_labels = [1] * len(train_pos_tweets)

    train_tweets = train_pos_tweets + train_neg_tweets
    train_labels = train_pos_labels + train_neg_labels
    #Shuffle
    train_tweets, train_labels = shuffle(train_tweets, train_labels, random_state=10)
    data = pd.DataFrame({'tweet': train_tweets, 'label': train_labels})

    X = list(data["tweet"])
    y = list(data["label"])
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.05,stratify=y,random_state=seed)
    X_train_set = pd.DataFrame({'tweet': X_train, 'label': y_train})
    X_val_set = pd.DataFrame({'tweet': X_val, 'label': y_val})

    X_test_set = pd.DataFrame({'tweet': test_tweets})


In [ ]:

embeddings = {}
with open("/content/drive/MyDrive/glove.twitter.27B.200d.txt", "r") as glove:
    for line in glove:
        line = line.split()
        word = line[0]
        vec = np.asarray(line[1:], "float32")
        embeddings[word] = vec
len(embeddings.keys())



1193514

In [ ]:
tokenizer = WhitespaceTokenizer()
def tokenize(df):
  df['tweet']=df['tweet'].apply(lambda tweet: tokenizer.tokenize(tweet))

In [ ]:
tokenize(X_train_set)
tokenize(X_val_set)
tokenize(X_test_set)

In [ ]:
def tweet_embedding(tokens):
    embedding = np.zeros(200)
    count = 0
    for token in tokens:
        if token in embeddings:
            embedding += embeddings[token]
            count = count+1
    # if count!=0:
    #   embedding = embedding/count
    return embedding

In [ ]:
x_train=np.vstack(X_train_set['tweet'].apply(lambda tokens: tweet_embedding(tokens)))
x_val=np.vstack(X_val_set['tweet'].apply(lambda tokens: tweet_embedding(tokens)))
x_test=np.vstack(X_test_set['tweet'].apply(lambda tokens: tweet_embedding(tokens)))

In [ ]:
# import torch
import logging
import sklearn
from sklearn.ensemble import RandomForestClassifier
import xgboost
from xgboost import XGBClassifier
# model = RandomForestClassifier(n_estimators=200,max_depth=10,min_samples_split=30,verbose=True,n_jobs=-1,random_state=seed)
model =  XGBClassifier(n_estimators =850,  subsample = 0.8, tree_method = "gpu_hist", objective='binary:logistic', use_label_encoder=False,seed = seed)
model.fit(x_train,y_train)

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=850, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(x_val)
accuracy = accuracy_score(y_val, y_pred)
print("Validation accuracy:", accuracy)

Validation accuracy: 0.7905429844294473


In [ ]:
y_test = model.predict(x_test)
y_preds = [-1 if val == 0 else 1 for val in y_test]
df = pd.DataFrame(y_preds, columns=["Predict[ion"])
df.index.name = "Id"
df.index += 1
df.to_csv("/content/drive/MyDrive/test_data_twitter_glove_final.csv")